# Imports and installations

In [ ]:
!pip install cartopy

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import  xarray as xr
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime,timedelta
import warnings
import math
from tqdm.auto import tqdm

In [ ]:
import warnings
warnings.filterwarnings('ignore')

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import cartopy.feature as cf
import cartopy.crs as ccrs
import cartopy.mpl.ticker as cticker

from cartopy.util import add_cyclic_point
from matplotlib import animation
from tqdm.auto import tqdm
from datetime import datetime,timedelta

from matplotlib.cm import get_cmap
warnings.filterwarnings("ignore")

# Temperature Anomaly




*   We calculated the **temperature anomalies** for each event using the dates from `HW_list_rolling_def.csv` for **pre-monsoon (MAM)** and **monsoon (JJAS)** seasons.  
*   The **climatology** for **pre-monsoon (MAM)** and **monsoon (JJAS)** season was derived from ERA5 multiple level (1000-900hpa) temperature data (1981–2010) and computed in :
    - `climatology_calculation_Temperature_(vertically_averaged).ipynb`


In [ ]:
# @title Heat date list
list_hw=pd.read_csv('/content/drive/MyDrive/AP_HW/Scripts-ll/HW_dates/data/HW_list_rolling_def.csv')
list_hw['Starting_Date'] = pd.to_datetime(list_hw["Starting_Date"])
list_hw['Ending_Date'] = pd.to_datetime(list_hw["Ending_Date"])

# separate hw for mam season
mam_hw_list =list_hw[(list_hw['Starting_Date'].dt.month >= 3) & (list_hw['Starting_Date'].dt.month <= 5)]
mam_hw_list.reset_index(inplace=True,drop=True)
mam_hw_list

In [ ]:
# @title Event data for composite analysis - temperature anomaly (MAM)

import matplotlib.pyplot as plt
# import cartopy.crs as ccrs
import numpy as np
import matplotlib.colors as colors

# List to store temperature anomalies for each event
anom_list = []

# Loop over MAM heatwave events
for i in tqdm(range(len(mam_hw_list['Starting_Date']))):
    # Extract event year and dates
    year = mam_hw_list['Starting_Date'].dt.year.iloc[i]
    start_time = str(mam_hw_list['Starting_Date'][i])
    end_time = str(mam_hw_list['Ending_Date'][i])

    # Adjust times to Bangladesh timezone
    start_time = str(np.datetime64(start_time) - np.timedelta64(6, 'h'))  # Subtract 6h
    end_time = str(np.datetime64(end_time) - np.timedelta64(6, 'h') + np.timedelta64(24, 'h'))  # Include end date
    time_range = slice(start_time, end_time)
    event_no = i + 1

    ###################################
    ############# temperature #########
    # Load temperature and climatology files
    ds_temp = xr.open_dataset(f"/content/drive/MyDrive/AP_HW/ERA5 Data/temperature_levels(1000-900)/temp(1000-900)-{year}.nc")
    temp_clim = xr.open_dataset('/content/drive/MyDrive/AP_HW/Scripts-ll/climatology/data/shum&temp/temp_climatology(1000-900)-mam.nc')

    # Select event period and compute mean
    ds_temp = ds_temp.sel(valid_time=time_range)
    ds_temp = ds_temp.mean(dim='valid_time', skipna=True)

    # Compute anomaly and assign event number
    temp_anom = ds_temp - temp_clim
    temp_anom = temp_anom.assign_coords(event_no=event_no)

    # Add to anomaly list
    anom_list.append(temp_anom)

# Concatenate all event anomalies
temp_events_anom_mam = xr.concat(anom_list, dim='event_no')

# Save to NetCDF
temp_events_anom_mam.to_netcdf('/content/drive/MyDrive/AP_HW/Scripts-ll/temp&shum/data/temp_events_anom-mam.nc')


In [ ]:
# @title Heat date list
list_hw=pd.read_csv('/content/drive/MyDrive/AP_HW/Scripts-ll/HW_dates/data/HW_list_rolling_def.csv')
list_hw['Starting_Date'] = pd.to_datetime(list_hw["Starting_Date"])
list_hw['Ending_Date'] = pd.to_datetime(list_hw["Ending_Date"])

# separate hw for jjas season
jjas_hw_list =list_hw[(list_hw['Starting_Date'].dt.month >= 6) & (list_hw['Starting_Date'].dt.month <= 9)]
jjas_hw_list.reset_index(inplace=True,drop=True)
jjas_hw_list


In [ ]:
# @title Event data for composite analysis - temperature anomaly (JJAS)

import matplotlib.pyplot as plt
# import cartopy.crs as ccrs
import numpy as np
import matplotlib.colors as colors

# List to store temperature anomalies for each event
anom_list = []

# Loop over JJAS heatwave events
for i in tqdm(range(len(jjas_hw_list['Starting_Date']))):
    # Extract event year and dates
    year = jjas_hw_list['Starting_Date'].dt.year.iloc[i]
    start_time = str(jjas_hw_list['Starting_Date'][i])
    end_time = str(jjas_hw_list['Ending_Date'][i])

    # Adjust times to Bangladesh timezone
    start_time = str(np.datetime64(start_time) - np.timedelta64(6, 'h'))  # Subtract 6h
    end_time = str(np.datetime64(end_time) - np.timedelta64(6, 'h') + np.timedelta64(24, 'h'))  # Include end date
    time_range = slice(start_time, end_time)
    event_no = i + 1

    ###################################
    ############# temperature #########
    # Load  temperature and climatology files
    ds_temp = xr.open_dataset(f"/content/drive/MyDrive/AP_HW/ERA5 Data/temperature_levels(1000-900)/temp(1000-900)-{year}.nc")
    temp_clim = xr.open_dataset('/content/drive/MyDrive/AP_HW/Scripts-ll/climatology/data/shum&temp/temp_climatology(1000-900)-jjas.nc')

    # Select event period and compute mean
    ds_temp = ds_temp.sel(valid_time=time_range)
    ds_temp = ds_temp.mean(dim='valid_time', skipna=True)

    # Compute anomaly and assign event number
    temp_anom = ds_temp - temp_clim
    temp_anom = temp_anom.assign_coords(event_no=event_no)

    # Add to anomaly list
    anom_list.append(temp_anom)

# Concatenate all event anomalies
temp_events_anom_jjas = xr.concat(anom_list, dim='event_no')

# Save to NetCDF
temp_events_anom_jjas.to_netcdf('/content/drive/MyDrive/AP_HW/Scripts-ll/temp&shum/data/temp_events_anom-jjas.nc')


# Specific humidity Anomaly



*   We calculated the **Specific humidity anomalies** for each event using the dates from `HW_list_rolling_def.csv` for **pre-monsoon (MAM)** and **monsoon (JJAS)** seasons.  
*   The **climatology** for **pre-monsoon (MAM)** and **monsoon (JJAS)** season was derived from ERA5 multiple level (1000-900hpa) Specific humidity data (1981–2010) and computed in :
    - `climatology_calculation_specific_humidity_(vertically_averaged).ipynb`


In [ ]:
# @title Heat date list
list_hw=pd.read_csv('/content/drive/MyDrive/AP_HW/Scripts-ll/HW_dates/data/HW_list_rolling_def.csv')
list_hw['Starting_Date'] = pd.to_datetime(list_hw["Starting_Date"])
list_hw['Ending_Date'] = pd.to_datetime(list_hw["Ending_Date"])

# separate hw for mam season
mam_hw_list =list_hw[(list_hw['Starting_Date'].dt.month >= 3) & (list_hw['Starting_Date'].dt.month <= 5)]
mam_hw_list.reset_index(inplace=True,drop=True)
mam_hw_list

In [ ]:
# @title Event data for composite analysis - specific humidity anomaly (MAM)

import matplotlib.pyplot as plt
# import cartopy.crs as ccrs
import numpy as np
import matplotlib.colors as colors

# List to store specific humidity anomalies for each event
anom_list = []

# Loop over MAM heatwave events
for i in tqdm(range(len(mam_hw_list['Starting_Date']))):
    # Extract event year and dates
    year = mam_hw_list['Starting_Date'].dt.year.iloc[i]
    start_time = str(mam_hw_list['Starting_Date'][i])
    end_time = str(mam_hw_list['Ending_Date'][i])

    # Adjust times to Bangladesh timezone
    start_time = str(np.datetime64(start_time) - np.timedelta64(6, 'h'))  # Subtract 6h
    end_time = str(np.datetime64(end_time) - np.timedelta64(6, 'h') + np.timedelta64(24, 'h'))  # Include end date
    time_range = slice(start_time, end_time)
    event_no = i + 1

    ###################################
    ############# specific humidity ####
    # Load event-specific specific humidity and climatology
    ds_shum = xr.open_dataset(f"/content/drive/MyDrive/AP_HW/ERA5 Data/specific_humidity/shum-{year}.nc")
    shum_clim = xr.open_dataset('/content/drive/MyDrive/AP_HW/Scripts-ll/climatology/data/shum&temp/shum_climatology(1000-900)-mam.nc')

    # Select event period and compute mean
    ds_shum = ds_shum.sel(valid_time=time_range)
    ds_shum = ds_shum.mean(dim='valid_time', skipna=True)

    # Compute anomaly and assign event number
    shum_anom = ds_shum - shum_clim
    shum_anom = shum_anom.assign_coords(event_no=event_no)

    # Add to anomaly list
    anom_list.append(shum_anom)

# Concatenate all event anomalies
shum_events_anom_mam = xr.concat(anom_list, dim='event_no')

# Save to NetCDF
shum_events_anom_mam.to_netcdf('/content/drive/MyDrive/AP_HW/Scripts-ll/temp&shum/data/shum_events_anom-mam.nc')


In [ ]:
# @title Heat date list
list_hw=pd.read_csv('/content/drive/MyDrive/AP_HW/Scripts-ll/HW_dates/data/HW_list_rolling_def.csv')
list_hw['Starting_Date'] = pd.to_datetime(list_hw["Starting_Date"])
list_hw['Ending_Date'] = pd.to_datetime(list_hw["Ending_Date"])

# separate hw for jjas season
jjas_hw_list =list_hw[(list_hw['Starting_Date'].dt.month >= 6) & (list_hw['Starting_Date'].dt.month <= 9)]
jjas_hw_list.reset_index(inplace=True,drop=True)
jjas_hw_list


In [ ]:
# @title Event data for composite analysis - specific humidity anomaly (JJAS)

import matplotlib.pyplot as plt
# import cartopy.crs as ccrs
import numpy as np
import matplotlib.colors as colors

# List to store specific humidity anomalies for each event
anom_list = []

# Loop over JJAS heatwave events
for i in tqdm(range(len(jjas_hw_list['Starting_Date']))):
    # Extract event year and dates
    year = jjas_hw_list['Starting_Date'].dt.year.iloc[i]
    start_time = str(jjas_hw_list['Starting_Date'][i])
    end_time = str(jjas_hw_list['Ending_Date'][i])

    # Adjust times to Bangladesh timezone
    start_time = str(np.datetime64(start_time) - np.timedelta64(6, 'h'))  # Subtract 6h
    end_time = str(np.datetime64(end_time) - np.timedelta64(6, 'h') + np.timedelta64(24, 'h'))  # Include end date
    time_range = slice(start_time, end_time)
    event_no = i + 1

    ###################################
    ############# specific humidity ####
    # Load event-specific specific humidity and climatology
    ds_shum = xr.open_dataset(f"/content/drive/MyDrive/AP_HW/ERA5 Data/specific_humidity/shum-{year}.nc")
    shum_clim = xr.open_dataset('/content/drive/MyDrive/AP_HW/Scripts-ll/climatology/data/shum&temp/shum_climatology(1000-900)-jjas.nc')

    # Select event period and compute mean
    ds_shum = ds_shum.sel(valid_time=time_range)
    ds_shum = ds_shum.mean(dim='valid_time', skipna=True)

    # Compute anomaly and assign event number
    shum_anom = ds_shum - shum_clim
    shum_anom = shum_anom.assign_coords(event_no=event_no)

    # Add to anomaly list
    anom_list.append(shum_anom)

# Concatenate all event anomalies
shum_events_anom_jjas = xr.concat(anom_list, dim='event_no')

# Save to NetCDF
shum_events_anom_jjas.to_netcdf('/content/drive/MyDrive/AP_HW/Scripts-ll/temp&shum/data/shum_events_anom-jjas.nc')


# Combine Plot


- We have plotted anomalies of vertically averaged **temperature and specific humidity**.

- Anomalies were calculated above in sections:  
  - `Temperature Anomaly`
`  
  - `Specific humidity Anomaly`
  


In [ ]:
levels_temp=np.arange(-5, 5+ 0.0001, 1)
levels_shum=np.arange(-0.003, 0.003 + 0.0001, 0.0005)
projection=ccrs.PlateCarree()
fig,axes=plt.subplots(ncols=2,nrows=2,subplot_kw={'projection':projection},figsize=(10, 25))

#  premonsoon temp
ax=axes.flat[0]
temp=xr.open_dataset('/content/drive/MyDrive/AP_HW/Scripts-ll/temp&shum/data/temp_events_anom-mam.nc')

# Spatial region
lat_range=slice(38,5)
lon_range=slice(60,98)
temp=temp.sel(latitude=lat_range,longitude=lon_range)
temp_anom=temp.mean(dim='event_no',skipna=True)
temp_anom=temp_anom.mean(dim='pressure_level',skipna=True)

latitude=temp_anom['latitude'].values
longitude=temp_anom['longitude'].values
contourf_temp=ax.contourf(longitude,latitude,temp_anom.t,levels=levels_temp,cmap='seismic',extend='both',transform=projection)
# cbar = plt.colorbar(contourf, ax=ax, orientation='horizontal', pad=0.01,fraction=0.06, label="${Pa}$")

ax.gridlines(draw_labels=['left','bottom'],visible=False)
ax.coastlines()
ax.set_title("(a) Temperature Anomalies - Pre-monsoon",fontsize=10)
ax.set_xlabel('Longitude')
ax.set_ylabel('Latitude')
ax.add_feature(cf.BORDERS.with_scale('10m'))

# premonsoon shum
ax=axes.flat[1]
shum=xr.open_dataset('/content/drive/MyDrive/AP_HW/Scripts-ll/temp&shum/data/shum_events_anom-mam.nc')

# Spatial region
lat_range=slice(38,5)
lon_range=slice(60,98)
shum=shum.sel(latitude=lat_range,longitude=lon_range)
shum_anom=shum.mean(dim='event_no',skipna=True)
shum_anom=shum_anom.mean(dim='pressure_level',skipna=True)

latitude=shum_anom['latitude'].values
longitude=shum_anom['longitude'].values
contourf_shum=ax.contourf(longitude,latitude,shum_anom.q,levels=levels_shum,cmap='seismic',extend='both',transform=projection)
# cbar = plt.colorbar(contourf, ax=ax, orientation='horizontal', pad=0.01,fraction=0.06, label="${Pa}$")

ax.gridlines(draw_labels=['bottom'],visible=False)
ax.coastlines()
ax.set_title("(b) Specific Humidity Anomalies - Pre-monsoon",fontsize=10)
ax.set_xlabel('Longitude')
ax.set_ylabel('Latitude')
ax.add_feature(cf.BORDERS.with_scale('10m'))


# monsoon temp
ax=axes.flat[2]
temp=xr.open_dataset('/content/drive/MyDrive/AP_HW/Scripts-ll/temp&shum/data/temp_events_anom-jjas.nc')

# Spatial region
lat_range=slice(38,5)
lon_range=slice(60,98)
temp=temp.sel(latitude=lat_range,longitude=lon_range)
temp_anom=temp.mean(dim='event_no',skipna=True)
temp_anom=temp_anom.mean(dim='pressure_level',skipna=True)

latitude=temp_anom['latitude'].values
longitude=temp_anom['longitude'].values
contourf_temp=ax.contourf(longitude,latitude,temp_anom.t,levels=levels_temp,cmap='seismic',extend='both',transform=projection)
# cbar = plt.colorbar(contourf, ax=ax, orientation='horizontal', pad=0.01,fraction=0.06, label="${Pa}$")

ax.gridlines(draw_labels=['left','bottom'],visible=False)
ax.coastlines()
ax.set_title("(c) Temperature Anomalies - Monsoon",fontsize=10)
ax.set_xlabel('Longitude')
ax.set_ylabel('Latitude')
ax.add_feature(cf.BORDERS.with_scale('10m'))

# monsoon shum
ax=axes.flat[3]
shum=xr.open_dataset('/content/drive/MyDrive/AP_HW/Scripts-ll/temp&shum/data/shum_events_anom-jjas.nc')

# Spatial region
lat_range=slice(38,5)
lon_range=slice(60,98)
shum=shum.sel(latitude=lat_range,longitude=lon_range)
shum_anom=shum.mean(dim='event_no',skipna=True)
shum_anom=shum_anom.mean(dim='pressure_level',skipna=True)

latitude=shum_anom['latitude'].values
longitude=shum_anom['longitude'].values
contourf_shum=ax.contourf(longitude,latitude,shum_anom.q,levels=levels_shum,cmap='seismic',extend='both',transform=projection)
# cbar = plt.colorbar(contourf, ax=ax, orientation='horizontal', pad=0.01,fraction=0.06, label="${Pa}$")

ax.gridlines(draw_labels=['bottom'],visible=False)
ax.coastlines()
ax.set_title("(d) Specific Humidity Anomalies - Monsoon",fontsize=10)
ax.set_xlabel('Longitude')
ax.set_ylabel('Latitude')
ax.add_feature(cf.BORDERS.with_scale('10m'))

#  contour levels for temperature
cbar_temp = plt.colorbar(contourf_temp, ax=axes, orientation='horizontal', pad=0.01, fraction=0.01, aspect=35, label="${Temperature}\ ({°C})$")
cbar_temp.set_ticks(levels_temp[::2])  # Set the colorbar ticks to match the levels used in contour plots
# Adjust colorbar position
cbar_temp.ax.set_position([0.125, 0.075, 0.38, 0.025]) # [left, bottom, width, height]

#  contour levels for shum
cbar_shum = plt.colorbar(contourf_shum, ax=axes, orientation='horizontal', pad=0.01, fraction=0.01, aspect=35, label="${Specific\ Humidity}\ ({kg}{kg}^{-1})$")
cbar_shum.set_ticks(levels_shum[::2])  # colorbar ticks
# Adjust colorbar position
cbar_shum.ax.set_position([0.525, 0.075, 0.38, 0.025]) # [left, bottom, width, height]


plt.subplots_adjust( wspace=0.05,hspace=-0.76,top=0.9)
plt.suptitle('Composite of 1000-900hPa Vertically Averaged Anomalies ', x=0.5, y=0.41,fontsize=11)
plt.show()

# Save the figure
fig.savefig("/content/drive/MyDrive/AP_HW/Scripts-ll/Manuscript codes/All_Figures/Fig_8_anomaly_of_vertically_averaged_sh_and_temp.jpg",
            dpi=300, format="jpg", bbox_inches="tight")

In [ ]:
print('done')